<a href="https://colab.research.google.com/github/KaifFaiyaz/Classification_gnn/blob/main/PyGeom1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.2 MB/s eta 0:00:00


In [4]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
#Data(edge_index=[2,4], x=[3,1])

In [5]:
print(data.keys())
print(data['x'])
print(data.num_nodes)
print(data.has_isolated_nodes)
print(data.has_self_loops)
print(data.is_directed)

['x', 'edge_index']
tensor([[-1.],
        [ 0.],
        [ 1.]])
3
<bound method BaseData.has_isolated_nodes of Data(x=[3, 1], edge_index=[2, 4])>
<bound method BaseData.has_self_loops of Data(x=[3, 1], edge_index=[2, 4])>
<bound method BaseData.is_directed of Data(x=[3, 1], edge_index=[2, 4])>


In [6]:
device=torch.device('cuda')
data=data.to(device)

In [7]:
from torch_geometric.datasets import TUDataset
dataset=TUDataset(root='/tmp/ENZYMES',name='ENZYMES')


Processing...
Done!


In [8]:
len(dataset)
print(dataset.num_classes,dataset.num_node_features)

6 3


In [9]:
train,test=dataset[:540],dataset[540:]

In [10]:
from torch_geometric.loader import  DataLoader

In [11]:
dataset_enz=TUDataset(root='/tmp/ENZYMES',name='ENZYMES',use_node_attr=True)
loader=DataLoader(dataset_enz,batch_size=32,shuffle=True)
for batch in loader:
  batch
  batch.num_graphs


In [13]:
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'])

dataset[0]
#>>> Data(pos=[2518, 3], y=[2518])

Extracting /tmp/ShapeNet/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Processing...
Done!


Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])

In [14]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6))

dataset[0]
#>>> Data(edge_index=[2, 15108], pos=[2518, 3], y=[2518])

/usr/local/lib/python3.10/dist-packages/torch_geometric/data/dataset.py:239: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, pass `force_reload=True` explicitly to reload the dataset.
  warnings.warn(


Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])

In [16]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6),
                    transform=T.RandomJitter(0.01))

dataset[0]
#>>> Data(edge_index=[2, 15108], pos=[2518, 3], y=[2518])

Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])

In [17]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
#Cora()

Processing...
Done!


In [18]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=GCNConv(dataset.num_node_features,16)
    self.conv2=GCNConv(16,dataset.num_classes)
  def forward(self,data):
    x,edge_index=data.x,data.edge_index
    x=self.conv1(x,edge_index)
    x=F.relu(x)
    x=F.dropout(x,training=self.training)
    x=self.conv2(x,edge_index)
    return F.log_softmax(x,dim=1)

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [20]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')
#>>> Accuracy: 0.8150

Accuracy: 0.7960
